In [1]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

# own libs
import sys
sys.path.append('//home//roman_vm//my_lib//')

from data_load import *

In [ ]:
f_list=['date', 'time', 'color', 'score', 'rating', 'opponent_rating', 'eco']

In [ ]:
work_df=df[df['rated']==True]
work_df=work_df[f_list]
work_df['datetime']=work_df['date']+' '+work_df['time']



c=pd.get_dummies(work_df['color'])
eco=pd.get_dummies(work_df['eco'])
work_df=pd.concat([work_df, c, eco], axis=1)




work_df=work_df.drop(columns=['time', 'datetime', 'color', 'eco'])

In [ ]:
hold=work_df[work_df['date']>='2019-08-01']
X=work_df[work_df['date']<'2019-08-01']
y=X['score']
X=X.drop(columns=['score', 'date'])

X=X.astype('int')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X.values, y.values, test_size=0.25, random_state=42)

In [ ]:
# строит график, считает метрики
def check(df, x, y1, y2, title):
    from sklearn.metrics import mean_squared_error
    print(title)
    df[[x, y1, y2]].set_index(x).plot(title=title)
    
    print('correlation: ', np.corrcoef(df[y1], df[y2])[0][1])
    
    print('stdev: ', mean_squared_error(df[y1], df[y2]))

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train, y_train)
lr_pred=lr.predict(X_test)

print('R^2 train:', lr.score(X_train, y_train))
print('R^2 test:', lr.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(lr.predict(X_test)))
print('MAE train', mean_absolute_error(lr.predict(X_train), y_train))
print('MAE test', mean_absolute_error(lr.predict(X_test), y_test))

# plt.plot(range(len(y_test)), y_test, lr.predict(X_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)

lr_pred=lr.predict(X_test)

print('R^2 train:', lr.score(X_train, y_train))
print('R^2 test:', lr.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(lr.predict(X_test)))
print('MAE train', mean_absolute_error(lr.predict(X_train), y_train))
print('MAE test', mean_absolute_error(lr.predict(X_test), y_test))

# plt.plot(range(len(y_test)), y_test, lr.predict(X_test))

In [ ]:
import xgboost
from sklearn.metrics import mean_absolute_error, mean_squared_error
xgb = xgboost.XGBRegressor()
xgb.fit(X_train, y_train)
print('R^2 train:', xgb.score(X_train, y_train))
print('R^2 test:', xgb.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(xgb.predict(X_test)))
print('MAE train', mean_absolute_error(xgb.predict(X_train), y_train))
print('MAE test', mean_absolute_error(xgb.predict(X_test), y_test))

In [ ]:
xgb_model_abs_testing = list()
xgb_model_abs_training = list()
rng = np.arange(1,151)
for i in rng:
    xgb = xgboost.XGBRegressor(n_estimators=i)
    xgb.fit(X_train, y_train)
    xgb.score(X_test, y_test)
    xgb_model_abs_testing.append(mean_absolute_error(xgb.predict(X_test), y_test))
    xgb_model_abs_training.append(mean_absolute_error(xgb.predict(X_train), y_train))
    
find_df=pd.DataFrame(np.arange(1,151), columns=['n_estimators']).set_index('n_estimators')
find_df['xgb_model_abs_testing_MAE']=xgb_model_abs_testing
find_df['xgb_model_abs_training_MAE']=xgb_model_abs_training

find_df.plot()

stat_best_n_estimators=np.argmin(find_df['xgb_model_abs_testing_MAE'])
ch_n_estimators=stat_best_n_estimators

In [ ]:
xgb_model_abs_testing = list()
xgb_model_abs_training = list()
rng = np.arange(0.05, 0.65, 0.05)
for i in rng:
    xgb = xgboost.XGBRegressor(n_estimators=ch_n_estimators, random_state=17, learning_rate=i)
    xgb.fit(X_train, y_train)
    xgb.score(X_test, y_test)
    xgb_model_abs_testing.append(mean_absolute_error(xgb.predict(X_test), y_test))
    xgb_model_abs_training.append(mean_absolute_error(xgb.predict(X_train), y_train))
    
find_df=pd.DataFrame(rng, columns=['learning_rate']).set_index('learning_rate')
find_df['xgb_model_abs_testing_MAE']=xgb_model_abs_testing
find_df['xgb_model_abs_training_MAE']=xgb_model_abs_training

find_df.plot()

stat_best_learning_rate=np.argmin(find_df['xgb_model_abs_testing_MAE'])
ch_learning_rate=0.15

In [ ]:
xgb_model_abs_testing = list()
xgb_model_abs_training = list()
rng = np.arange(1, 11)
for i in rng:
    xgb = xgboost.XGBRegressor(n_estimators=ch_n_estimators, random_state=17, learning_rate=ch_learning_rate, max_depth=i)
    xgb.fit(X_train, y_train)
    xgb.score(X_test, y_test)
    xgb_model_abs_testing.append(mean_absolute_error(xgb.predict(X_test), y_test))
    xgb_model_abs_training.append(mean_absolute_error(xgb.predict(X_train), y_train))
    
find_df=pd.DataFrame(rng, columns=['max_depth']).set_index('max_depth')
find_df['xgb_model_abs_testing_MAE']=xgb_model_abs_testing
find_df['xgb_model_abs_training_MAE']=xgb_model_abs_training

find_df.plot()

stat_best_max_depth=np.argmin(find_df['xgb_model_abs_testing_MAE'])
ch_max_depth=2

In [ ]:
ch_max_depth=2

In [ ]:
import xgboost
from sklearn.metrics import mean_absolute_error, mean_squared_error
xgb = xgboost.XGBRegressor(n_estimators=ch_n_estimators, random_state=17, learning_rate=ch_learning_rate, max_depth=ch_max_depth)
xgb.fit(X_train, y_train)
print('R^2 train:', xgb.score(X_train, y_train))
print('R^2 test:', xgb.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(xgb.predict(X_test)))
print('MAE train', mean_absolute_error(xgb.predict(X_train), y_train))
print('MAE test', mean_absolute_error(xgb.predict(X_test), y_test))

In [ ]:
res_test=pd.DataFrame(X_test, columns=X.columns)
res_test['fact']=y_test
res_test['predict']=xgb.predict(X_test)


sh=res_test.sample(150).reset_index()
sh['ind']=sh.index

In [ ]:
check(sh, 'ind', 'fact', 'predict', 'gg')

In [ ]:
features_df = pd.DataFrame(data=xgb.feature_importances_.reshape(1, -1), columns=X.columns).sort_values(axis=1, by=[0], ascending=False).T

In [ ]:
features_df

In [ ]:
res_test.groupby('month').mean()[['fact']]

In [ ]:
res_test.groupby('hour').mean()[['fact']]